## \[Сбор данных\] Лабораторная работа по определению зависимости сопротивления материала от его температуры.

Дисклеймер: для работы необходима работающая pyvisa.

Данный ноутбук является заменой для нелюбимого мной LabView. Реализовано две вариации четырехточечной схема измерения с шунтом: с использованием платинового термосопротивления (или любого иного термосопротивления) и с использованием термопары.

Перед началом работы введите корректные параметры прибора измерения температуры и шунта, корректные адреса приборов и название файла, в который будут записываться данные.

К сожалению, нормального выхода из цикла измерения я пока не придумал, поэтому используйте keyboard interrupt. Частично данные сохраняются автоматически, чтобы их сохранить полностью не забудьте активировать последнюю ячейку.

[f3ss1](https://github.com/f3ss1)

In [1]:
import pyvisa as visa
import time
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
# Temperature sensor config (Pt sensor)
alpha = 4
R_0 = 1000
T_0 = 273

# Shunt config
R_shunt = 0.0075

In [2]:
# Temperature sensor config (thermocouple) -- requires more precise config
alpha = 0.15 / 4000
V_0 = 1.33 / 1000
T_0 = 337

# Shunt config
R_shunt = 0.0075

In [3]:
# Getting devices addresses

rm = visa.ResourceManager()
rm.list_resources()

('USB0::0x2A8D::0xB318::MY58290011::INSTR',
 'USB0::0x2A8D::0xB318::MY58320025::INSTR',
 'USB0::0x2A8D::0xB318::MY58320045::INSTR',
 'ASRL1::INSTR',
 'ASRL2::INSTR')

In [4]:
temp_inst = rm.open_resource('USB0::0x2A8D::0xB318::MY58320045::INSTR', write_termination= '\n', read_termination='\n')
shunt_inst = rm.open_resource('USB0::0x2A8D::0xB318::MY58320025::INSTR', write_termination= '\n', read_termination='\n')
sample_inst = rm.open_resource('USB0::0x2A8D::0xB318::MY58290011::INSTR', write_termination= '\n', read_termination='\n')

In [7]:
# Setting temperature device in case of Pt sensor
temp_inst.write('CONF:RES')
time.sleep(1)
temp_inst.write('*OPC?')
time.sleep(1)
temp_inst.read()

'1'

In [5]:
# Setting temperature device in case of thermocouple
temp_inst.write('CONF:voltage:dc')
time.sleep(1)
temp_inst.write('*OPC?')
time.sleep(1)
temp_inst.read()

'1'

In [6]:
# Setting shunt device
shunt_inst.write('CONF:voltage:dc')
time.sleep(1)
shunt_inst.write('*OPC?')
time.sleep(1)
shunt_inst.read()

'1'

In [7]:
# Setting sample device
sample_inst.write('CONF:voltage:dc')
time.sleep(1)
sample_inst.write('*OPC?')
time.sleep(1)
sample_inst.read()

'1'

In [8]:
def get_R(device):
    device.write('INIT')
    device.write('FETC?')
    time.sleep(0.1)
    return float(device.read())

In [9]:
def get_V(device):
    device.write('READ?')
    time.sleep(0.1)
    return float(device.read())

In [10]:
data = pd.DataFrame(columns=['Time', 'V_sa', 'V_sh', 'I', 'R_sa', 'T'], dtype=np.float)
filename = 'Nichrome heat series 1.csv' # Insert file name here

,Time,V_sa,V_sh,I,R_sa,Temp_R


In [15]:
# In case Pt sensor is used
zero_time = time.process_time()
f = open('reference.txt', 'a')
i = 0
while True:
    cur_time = time.process_time() - zero_time
    temp_R = get_R(temp_inst)
    T = (temp_R - R_0) / alpha + T_0
    V_shunt = get_V(shunt_inst)
    I = V_shunt / R_shunt
    V_sample = get_V(sample_inst)
    R_sample = V_sample / I
    print('Current T: {}\nCurrent R: {}'.format(T, R_sample))
    data = data.append({'Time':cur_time, 'V_sa': V_sample, 'V_sh': V_shunt, 'I':I, 'R_sa':R_sample, 'T':T},ignore_index=True)
    f.write(str(T) + ', ' + str(R_sample))
    if (i % 20 == 0):
        data.to_csv(filename, sep=';')
    i += 1

NameError: name 'R_0' is not defined

In [11]:
# In case thermocouple is used

zero_time = time.process_time()
i = 0
while True:
    cur_time = time.process_time() - zero_time
    V_temp = get_V(temp_inst)
    T = (V_temp - V_0) / alpha + T_0
    V_shunt = get_V(shunt_inst)
    I = V_shunt / R_shunt
    V_sample = get_V(sample_inst)
    R_sample = V_sample / I
    print('Current T: {}\nCurrent R: {}'.format(T, R_sample))
    data = data.append({'Time':cur_time, 'V_sa': V_sample, 'V_sh': V_shunt, 'I':I, 'R_sa':R_sample, 'T':T},ignore_index=True)
    if (i % 20 == 0):
        data.to_csv(filename, sep=';')
    i += 1

Current R: 95.39774701233762
-8.182470000000098e-06
Current T: 336.7818008
Current R: 95.24745683243188
-7.491870000000088e-06
Current T: 336.8002168
Current R: 95.51784637560509
-5.3361600000001105e-06
Current T: 336.8577024
Current R: 95.48181164246475
-5.626699999999917e-06
Current T: 336.8499546666667
Current R: 95.55241392073982
-4.348869999999932e-06
Current T: 336.88403013333334
Current R: 95.73060673648921
-3.946660000000079e-06
Current T: 336.89475573333334
Current R: 95.56908182521049
-1.435460000000048e-06
Current T: 336.96172106666666
Current R: 95.67195977315028
-3.9146000000008993e-07
Current T: 336.9895610666667
Current R: 95.82815346594764
2.7685300000000485e-06
Current T: 337.07382746666667
Current R: 95.64723779890888
3.917489999999977e-06
Current T: 337.1044664
Current R: 96.0047361937787
7.521219999999905e-06
Current T: 337.20056586666664
Current R: 95.78254915779293
9.407209999999933e-06
Current T: 337.25085893333335
Current R: 95.86489126374327
1.1555950000000006e

KeyboardInterrupt: 

In [12]:
data.to_csv(filename, sep=';')